In [2]:
import spiceypy as spice
import numpy as np

# Load the required kernels (including DSK)
dsk_file = "/media/mglos/HDD_8TB1/SPICE/moon_lowres.bds"
ids = spice.dskobj(dsk_file)

print("Objects in DSK file:", ids)

body_id = 301  # Moon
surfaces = spice.dsksrf(dsk_file, body_id)

print("Available surfaces for body:", surfaces)

# Open the DSK file
handle = spice.dasopr(dsk_file)

# Begin traversing DLA (Data Linked Array) segments
dladsc = spice.dlabfs(handle)  # This returns a single SpiceDLADescr object

if dladsc is not None:
    # Retrieve DSK descriptor
    dskdsc = spice.dskgd(handle, dladsc)
    print("DSK Descriptor:", dskdsc)
else:
    print("No valid DLA descriptor found in the DSK file.")

spice.dascls(handle)


Objects in DSK file: <SpiceCell dtype = 2, length = 0, size = 10000, card = 1, is_set = 1, adjust = 0, init = 1, base = 572726800, data = 572726824>
Available surfaces for body: <SpiceCell dtype = 2, length = 0, size = 10000, card = 1, is_set = 1, adjust = 0, init = 1, base = 572766832, data = 572766856>
DSK Descriptor: <spiceypy.utils.support_types.SpiceDSKDescr object at 0x7971602511c0>


In [3]:
# Open the DSK file for read access
handle = spice.dasopr(dsk_file)
dladsc = spice.dlabfs(handle)

if dladsc:
    # Get the number of vertices and plates
    nv, _np = spice.dskz02(handle, dladsc)
    print(f"Number of vertices: {nv}")
    print(f"Number of plates: {_np}")

    # Read all vertices
    vertices = []
    for start in range(1, nv + 1, 1000):
        n = min(1000, nv - start + 1)
        verts = spice.dskv02(handle, dladsc, start, n)
        vertices.extend(verts)

    # Read all plates
    plates = []
    for start in range(1, _np + 1, 1000):
        n = min(1000, _np - start + 1)
        pls = spice.dskp02(handle, dladsc, start, n)
        plates.extend(pls)

    # Convert to numpy arrays for easier manipulation
    vertices = np.array(vertices)
    plates = np.array(plates) - 1  # Convert to 0-based indexing for Python

    print("Sample vertices:", vertices[:5])
    print("Sample plates:", plates[:5])
else:
    print("No segment found in DSK file.")

# Close the DSK file
spice.dascls(handle)

SpiceFILENOTFOUND: 
================================================================================

Toolkit version: CSPICE_N0067

SPICE(FILENOTFOUND) --

The file 'path_to_dsk_file.bds' does not exist.

dasopr_c --> DASOPR --> ZZDDHOPN

================================================================================

In [5]:
x = spice.dlabfs(handle)
dir(x)

['__class__',
 '__ctypes_from_outparam__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_b_base_',
 '_b_needsfree_',
 '_bwdptr',
 '_cbase',
 '_csize',
 '_dbase',
 '_dsize',
 '_fields_',
 '_fwdptr',
 '_ibase',
 '_isize',
 '_objects',
 'bwdptr',
 'cbase',
 'csize',
 'dbase',
 'dsize',
 'fwdptr',
 'ibase',
 'isize']

In [10]:
import spiceypy as spice
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import os

# 🔹 Define input DSK file
DSK_FILE = "/media/mglos/HDD_8TB1/LOLA_DSK/Lunar_4_percent.dsk"
CACHE_DIR = "/media/mglos/HDD_8TB1/LOLA_DSK/CACHE/"
os.makedirs(CACHE_DIR, exist_ok=True)

# 🔹 Load SPICE Kernels
print("Loading SPICE Kernels...")
spice.furnsh(DSK_FILE)
print("✅ Kernels Loaded.")

# 🔹 Extract Shape Model Data
print("Extracting DSK data...")
dladsc = spice.dlabfs(DSK_FILE)  # Find first segment
nv, vrtces = spice.dskv02(DSK_FILE, dladsc, 1, 10000000)  # Load vertices
np, plates = spice.dskp02(DSK_FILE, dladsc, 1, 10000000)  # Load plates
print(f"✅ Loaded {nv} vertices and {np} plates.")

# 🔹 Convert Vertices to Numpy Array
vrtces = np.array(vrtces)
lon, lat, radius = spice.reclat(vrtces.T)
lon, lat, radius = np.degrees(lon), np.degrees(lat), radius

# 🔹 3D Globe Visualization
print("Generating 3D Globe Visualization...")
fig_3d = go.Figure(data=[
    go.Scatter3d(
        x=vrtces[:, 0], y=vrtces[:, 1], z=vrtces[:, 2],
        mode='markers',
        marker=dict(size=1, color=radius, colorscale='Viridis', opacity=1)
    )
])
fig_3d.update_layout(
    title="3D Lunar Shape Model",
    scene=dict(
        xaxis_title="X (km)",
        yaxis_title="Y (km)",
        zaxis_title="Z (km)",
        aspectmode='data'
    )
)
fig_3d.show()

# 🔹 2D Cylindrical Projection Map
print("Generating 2D Cylindrical Projection...")
fig, ax = plt.subplots(figsize=(12, 6))
sc = ax.scatter(lon, lat, c=radius, cmap='viridis', s=0.5)
ax.set_xlabel("Longitude (°)")
ax.set_ylabel("Latitude (°)")
ax.set_title("Simple Cylindrical Projection of Lunar Surface")
plt.colorbar(sc, label="Elevation (km)")
plt.show()

# Cleanup
spice.kclear()
print("✅ Visualization Completed.")


Loading SPICE Kernels...


SpiceDASFILEREADFAILED: 
================================================================================

Toolkit version: CSPICE_N0067

SPICE(DASFILEREADFAILED) --

Could not read DAS integer record. File = /media/mglos/HDD_8TB1/LOLA_DSK/Lunar_4_percent.dsk Record number = 2. IOSTAT = -1.

furnsh_c --> FURNSH --> ZZLDKER --> ZZDSKLSF --> DASOPR --> ZZDASGRI

================================================================================

In [1]:
import sys
import importlib
from datetime import timedelta
import spiceypy
from pprint import pprint
from astropy.time import TimeDelta

sys.path.insert(0, "/home/mglos/skola/DIP/LavaTubeSniffer")
from src.scripts.SPICE.config import TIME_STEP, DIVINER_INSTRUMENT_IDS
from src.mongo.interface import Sessions
from src.scripts.SPICE import utils

importlib.reload(utils)

DIVINER_INSTRUMENT_IDS = [-85211,-85212,-85213,-85214,-85215,-85216,-85221,-85222,-85223]
sweep_iterator = utils.SweepIterator()

computation_start = sweep_iterator.min_loaded_time
computation_end = sweep_iterator.max_loaded_time

computation_timedelta = timedelta(days=1000)
computation_now = computation_start + computation_timedelta

sweep_iterator.initiate_sweep(computation_now)

Loading static SPICE kernels: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 77/77 [00:06<00:00, 11.24it/s]


In [2]:
spiceypy.furnsh("/media/mglos/HDD_8TB1/SPICE/moon_pa_de440_200625.bpc")
spiceypy.furnsh("/media/mglos/HDD_8TB1/SPICE/pck00011.tpc")
spiceypy.furnsh("/media/mglos/HDD_8TB1/SPICE/de441_part-1.bsp")
spiceypy.furnsh("/media/mglos/HDD_8TB1/SPICE/de441_part-2.bsp")
spiceypy.furnsh("/media/mglos/HDD_8TB1/LOLA_DSK/Lunar_LDEM_118m.dsk")

In [ ]:
# dsk_public = "/media/mglos/HDD_8TB1/SPICE/moon_lowres.bds"
# dsk_my = "/media/mglos/HDD_8TB1/LOLA_DSK/Lunar_LDEM_118m.dsk"

In [20]:
# import spiceypy as spice
# import numpy as np

# # Open both DSK files
# handle1 = spice.dasopr(dsk_my)
# handle2 = spice.dasopr(dsk_public)

# # Read the first segment of each DSK
# dladsc1 = spice.dlabfs(handle1)
# dladsc2 = spice.dlabfs(handle2)

# # Get vertex and plate counts
# nv1, np1 = spice.dskz02(handle1, dladsc1)
# nv2, np2 = spice.dskz02(handle2, dladsc2)

# print(f"My DSK: {nv1} vertices, {np1} plates")
# print(f"NASA DSK: {nv2} vertices, {np2} plates")

# # Read vertices and plates
# vertices1 = np.array(spice.dskv02(handle1, dladsc1, 1, nv1))
# plates1 = np.array(spice.dskp02(handle1, dladsc1, 1, np1)) - 1  # Convert to 0-based indexing

# vertices2 = np.array(spice.dskv02(handle2, dladsc2, 1, nv2))
# plates2 = np.array(spice.dskp02(handle2, dladsc2, 1, np2)) - 1

# # Close files
# spice.dascls(handle1)
# spice.dascls(handle2)

# # Compare Min/Max values
# print(f"My DSK X range: {vertices1[:,0].min()} to {vertices1[:,0].max()}")
# print(f"NASA DSK X range: {vertices2[:,0].min()} to {vertices2[:,0].max()}")

# print(f"My DSK Y range: {vertices1[:,1].min()} to {vertices1[:,1].max()}")
# print(f"NASA DSK Y range: {vertices2[:,1].min()} to {vertices2[:,1].max()}")

# print(f"My DSK Z range: {vertices1[:,2].min()} to {vertices1[:,2].max()}")
# print(f"NASA DSK Z range: {vertices2[:,2].min()} to {vertices2[:,2].max()}")

# # Compute vertex mean distance
# mean_dist = np.linalg.norm(vertices1.mean(axis=0) - vertices2.mean(axis=0))
# print(f"Mean position difference: {mean_dist} km")


My DSK: 423660 vertices, 844560 plates
NASA DSK: 1036802 vertices, 2073600 plates
My DSK X range: -1741.5482129372003 to 1736.629830636686
NASA DSK X range: -1741.2182901571475 to 1736.6702340361821
My DSK Y range: -1737.9811480796482 to 1737.8392936213113
NASA DSK Y range: -1739.6183206848473 to 1734.4791794582102
My DSK Z range: -1742.9273975305987 to 1742.4274441726163
NASA DSK Z range: -1739.0668613194475 to 1737.6348647273535
Mean position difference: 1.281602591918171 km


In [ ]:
# import spiceypy as spice
# import numpy as np

# # 🔹 Load both DSK files
# dsk1 = "/path/to/your_generated.dsk"
# dsk2 = "/path/to/NASA_reference.dsk"



# handles = []
# handles.append(spice.dasopr(dsk1))
# handles.append(spice.dasopr(dsk2))

# # 🔹 Get segment information
# dladsc1 = spice.dlabfs(handles[0])  # First segment
# dladsc2 = spice.dlabfs(handles[1])

# # 🔹 Define test query points (lon, lat in radians)
# test_lonlat = [
#     (np.radians(-90), np.radians(0)),  # Prime meridian, equator
#     (np.radians(0), np.radians(45)),   # 45° North
#     (np.radians(90), np.radians(-30)), # 30° South, 90° East
# ]

# # 🔹 Convert lon/lat into ray vectors (for DSK query)
# target_body = 301  # Moon
# frame = "MOON_ME"
# abcorr = "NONE"
# observer = 0  # Assume we are querying from center of Moon
# radii = [1737400, 1737400, 1737400]  # Moon reference ellipsoid

# for lon, lat in test_lonlat:
#     raydir = spice.srfrec(target_body, lon, lat, radii[0])  # Convert to XYZ vector

#     # 🔹 Query both DSKs at the same location
#     xpt1, found1 = spice.dskxv(False, handles[0], dladsc1, frame, raydir, observer, abcorr)
#     xpt2, found2 = spice.dskxv(False, handles[1], dladsc2, frame, raydir, observer, abcorr)

#     if found1 and found2:
#         print(f"📍 Query at Lon: {np.degrees(lon)}, Lat: {np.degrees(lat)}")
#         print(f"  My DSK Elevation: {xpt1[-1]:.3f} km")
#         print(f"  NASA DSK Elevation: {xpt2[-1]:.3f} km")
#         print(f"  Difference: {abs(xpt1[-1] - xpt2[-1]):.3f} km\n")

# # 🔹 Close the DSK files
# spice.dascls(handles[0])
# spice.dascls(handles[1])


In [17]:
# print(f"My point {vertices1[0]}; other point {vertices2[0]}")

My point [-8.77905235e-13 -1.73732000e+03 -1.75581047e-12]; other point [   0.            0.         1737.29807847]


In [18]:
# print(f"My point {vertices1[-1]}; other point {vertices2[-1]}")

My point [-1.06393028e-13  1.73753000e+03  2.12786055e-13]; other point [    0.            0.        -1737.4311316]


In [19]:
# print(f"My point {vertices1[int(len(vertices1)/2)]}; other point {vertices2[int(len(vertices2)/2)]}")

My point [-1.74059981e+03  5.95672867e+00 -1.75913549e-12]; other point [1736.67023404    3.78883271   -3.78884173]


In [2]:
import spiceypy as spice
import spiceypy
spiceypy.tkvrsn('TOOLKIT')

import sys
import importlib
from datetime import timedelta

from pprint import pprint
from astropy.time import TimeDelta

sys.path.insert(0, "/home/mglos/skola/DIP/LavaTubeSniffer")
from src.scripts.SPICE.config import TIME_STEP, DIVINER_INSTRUMENT_IDS
from src.mongo.interface import Sessions
from src.scripts.SPICE import utils

importlib.reload(utils)

DIVINER_INSTRUMENT_IDS = [-85211,-85212,-85213,-85214,-85215,-85216,-85221,-85222,-85223]
sweep_iterator = utils.SweepIterator()

computation_start = sweep_iterator.min_loaded_time
computation_end = sweep_iterator.max_loaded_time

computation_timedelta = timedelta(days=1000)
computation_now = computation_start + computation_timedelta

sweep_iterator.initiate_sweep(computation_now)
spiceypy.furnsh("/media/mglos/HDD_8TB1/SPICE/moon_pa_de440_200625.bpc")
spiceypy.furnsh("/media/mglos/HDD_8TB1/SPICE/pck00011.tpc")
spiceypy.furnsh("/media/mglos/HDD_8TB1/SPICE/de441_part-1.bsp")
spiceypy.furnsh("/media/mglos/HDD_8TB1/SPICE/de441_part-2.bsp")
spiceypy.furnsh("/media/mglos/HDD_8TB1/SPICE/moon_lowres.bds")


points = Sessions.get_all_pits_points()

from astropy.time import Time
import spiceypy as spice

time_utc = computation_start.utc.iso
computation_now = computation_start + computation_timedelta
et = spice.str2et(computation_now.utc.iso)

# Store FOV parameters for each channel
fov_data = {}
for naif_id in DIVINER_INSTRUMENT_IDS:
    # Get FOV shape, boresight, and boundary vectors
    shape, frame, boresight, n, bounds = spice.getfov(naif_id, room=100)
    fov_data[naif_id] = {
        "shape": shape,
        "boresight": boresight,
        "bounds": bounds,
        "frame": frame,
    }

for fov in fov_data.values():
    print(fov['boresight'])


spice.sincpt('DSK/UNPRIORITIZED', 'MOON', et, "MOON_ME", 'CN+S', 'LUNAR RECONNAISSANCE ORBITER', 'LRO_DLRE_FP_A', fov_data[-85222]['bounds'][1])
#spice.spkpos('MOON', et, 'J2000', 'NONE', 'LRO')

Loading static SPICE kernels: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 77/77 [00:07<00:00, 10.71it/s]


[-4.70541289e-04  3.48390510e-02 -9.99392825e-01]
[-4.70508474e-04  2.11845630e-02 -9.99775471e-01]
[-2.62134746e-04  6.20422281e-03 -9.99980719e-01]
[ 3.15274790e-05 -7.84337655e-03 -9.99969240e-01]
[ 1.63894230e-04 -2.13255346e-02 -9.99772571e-01]
[ 1.54082347e-04 -3.30422543e-02 -9.99453944e-01]
[-6.99791693e-04  3.68294807e-02 -9.99321320e-01]
[-8.44512893e-04  4.23016546e-03 -9.99990696e-01]
[-0.00116639 -0.03207335 -0.99948484]


(array([-308.60738153, -421.4701902 , 1656.99854354]),
 386484690.3960072,
 array([  28.20908259,   50.81622861, -185.34341701]))

In [8]:
points = Sessions.get_all_pits_points()

from astropy.time import Time
import spiceypy as spice

time_utc = computation_start.utc.iso
computation_now = computation_start + computation_timedelta
et = spice.str2et(computation_now.utc.iso)

# Store FOV parameters for each channel
fov_data = {}
for naif_id in DIVINER_INSTRUMENT_IDS:
    # Get FOV shape, boresight, and boundary vectors
    shape, frame, boresight, n, bounds = spice.getfov(naif_id, room=100)
    fov_data[naif_id] = {
        "shape": shape,
        "boresight": boresight,
        "bounds": bounds,
        "frame": frame,
    }

for fov in fov_data.values():
    print(fov['boresight'])


spice.sincpt('DSK/UNPRIORITIZED', 'MOON', et, "MOON_ME", 'CN+S', 'LUNAR RECONNAISSANCE ORBITER', 'LRO_DLRE_FP_A', diviner_bounds[0])
#spice.spkpos('MOON', et, 'J2000', 'NONE', 'LRO')

[-4.70541289e-04  3.48390510e-02 -9.99392825e-01]
[-4.70508474e-04  2.11845630e-02 -9.99775471e-01]
[-2.62134746e-04  6.20422281e-03 -9.99980719e-01]
[ 3.15274790e-05 -7.84337655e-03 -9.99969240e-01]
[ 1.63894230e-04 -2.13255346e-02 -9.99772571e-01]
[ 1.54082347e-04 -3.30422543e-02 -9.99453944e-01]
[-6.99791693e-04  3.68294807e-02 -9.99321320e-01]
[-8.44512893e-04  4.23016546e-03 -9.99990696e-01]
[-0.00116639 -0.03207335 -0.99948484]


(array([-294.02761628, -423.39811147, 1659.68186213]),
 386484690.39600843,
 array([  42.78848705,   48.88795965, -182.65877747]))

In [4]:
fov_data[-85222]['bounds'][1]

array([-3.55946048e-02,  8.25994569e-04, -9.99365970e-01])

In [5]:
# Load the instrument data. TODO: Maybe 
diviner_bounds = []
for naif_id in DIVINER_INSTRUMENT_IDS:
    # Get FOV shape, boresight, and boundary vectors
    shape, frame, boresight, n, bounds = spice.getfov(naif_id, room=100)
    diviner_bounds.extend(bounds.copy())

In [7]:
diviner_bounds[0]

array([ 0.03480622,  0.03204172, -0.9988803 ])

In [9]:
fov_data

{-85211: {'shape': 'POLYGON',
  'boresight': array([-4.70541289e-04,  3.48390510e-02, -9.99392825e-01]),
  'bounds': array([[ 0.03480622,  0.03204172, -0.9988803 ],
         [ 0.03480597,  0.03763517, -0.9986852 ],
         [-0.03560505,  0.03814578, -0.99863766],
         [-0.03560531,  0.03255259, -0.99883562]]),
  'frame': 'LRO_DLRE_FP_A'},
 -85212: {'shape': 'POLYGON',
  'boresight': array([-4.70508474e-04,  2.11845630e-02, -9.99775471e-01]),
  'bounds': array([[ 0.03491917,  0.02018487, -0.99918628],
         [ 0.03491912,  0.0221833 , -0.99914391],
         [-0.03533702,  0.02205985, -0.99913195],
         [-0.03533707,  0.02006145, -0.99917407]]),
  'frame': 'LRO_DLRE_FP_A'},
 -85213: {'shape': 'POLYGON',
  'boresight': array([-2.62134746e-04,  6.20422281e-03, -9.99980719e-01]),
  'bounds': array([[ 0.03506375,  0.00400503, -0.99937705],
         [ 0.03506371,  0.00840245, -0.99934976],
         [-0.03494741,  0.00833187, -0.99935442],
         [-0.03494744,  0.00393446, -0.9993

In [ ]:
import spiceypy as spice
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import os

# 🔹 Define input DSK file
DSK_FILE = "/media/mglos/HDD_8TB1/LOLA_DSK/Lunar_4_percent.dsk"
CACHE_DIR = "/media/mglos/HDD_8TB1/LOLA_DSK/CACHE/"
os.makedirs(CACHE_DIR, exist_ok=True)

# 🔹 Load SPICE Kernels
print("Loading SPICE Kernels...")
spice.furnsh(DSK_FILE)
print("✅ Kernels Loaded.")

# 🔹 Extract Shape Model Data
print("Extracting DSK data...")
dladsc = spice.dlabfs(DSK_FILE)  # Find first segment
nv, vrtces = spice.dskv02(DSK_FILE, dladsc, 1, 10000000)  # Load vertices
np, plates = spice.dskp02(DSK_FILE, dladsc, 1, 10000000)  # Load plates
print(f"✅ Loaded {nv} vertices and {np} plates.")

# 🔹 Convert Vertices to Numpy Array
vrtces = np.array(vrtces)
lon, lat, radius = spice.reclat(vrtces.T)
lon, lat, radius = np.degrees(lon), np.degrees(lat), radius

# 🔹 3D Globe Visualization
print("Generating 3D Globe Visualization...")
fig_3d = go.Figure(data=[
    go.Scatter3d(
        x=vrtces[:, 0], y=vrtces[:, 1], z=vrtces[:, 2],
        mode='markers',
        marker=dict(size=1, color=radius, colorscale='Viridis', opacity=1)
    )
])
fig_3d.update_layout(
    title="3D Lunar Shape Model",
    scene=dict(
        xaxis_title="X (km)",
        yaxis_title="Y (km)",
        zaxis_title="Z (km)",
        aspectmode='data'
    )
)
fig_3d.show()

# 🔹 2D Cylindrical Projection Map
print("Generating 2D Cylindrical Projection...")
fig, ax = plt.subplots(figsize=(12, 6))
sc = ax.scatter(lon, lat, c=radius, cmap='viridis', s=0.5)
ax.set_xlabel("Longitude (°)")
ax.set_ylabel("Latitude (°)")
ax.set_title("Simple Cylindrical Projection of Lunar Surface")
plt.colorbar(sc, label="Elevation (km)")
plt.show()

# Cleanup
spice.kclear()
print("✅ Visualization Completed.")
